## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Peto,MX,20.1333,-88.9167,71.17,100,100,1.48,overcast clouds
1,1,Albany,US,42.6001,-73.9662,32.07,85,3,3.76,clear sky
2,2,Dikson,RU,73.5069,80.5464,-8.93,95,95,6.02,overcast clouds
3,3,Hobart,AU,-42.8794,147.3294,59.11,76,75,1.01,broken clouds
4,4,Hermanus,ZA,-34.4187,19.2345,67.89,73,95,17.67,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Avarua,CK,-21.2078,-159.7750,77.05,78,20,6.91,few clouds
8,8,Hervey Bay,AU,-25.2986,152.8535,82.04,60,0,10.69,clear sky
11,11,Qui Nhon,VN,13.7667,109.2333,81.25,75,23,8.41,few clouds
15,15,Camacari,BR,-12.6975,-38.3242,78.57,94,16,3.89,few clouds
21,21,Bambous Virieux,MU,-20.3428,57.7575,82.65,61,75,16.11,broken clouds
23,23,Busselton,AU,-33.6500,115.3333,79.09,40,19,9.04,few clouds
27,27,Raga,SS,8.4596,25.6780,84.70,20,6,9.42,clear sky
29,29,Liloy,PH,8.1144,122.6594,83.19,78,100,2.95,heavy intensity rain
37,37,Lilongwe,MW,-13.9669,33.7873,82.60,44,0,5.75,clear sky
40,40,Atuona,PF,-9.8000,-139.0333,78.17,73,15,15.73,few clouds


In [5]:
# 4a. Determine if there are any empty rows
preferred_cities_df.count()

City_ID                204
City                   204
Country                200
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                200
City                   200
Country                200
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,
8,Hervey Bay,AU,82.04,clear sky,-25.2986,152.8535,
11,Qui Nhon,VN,81.25,few clouds,13.7667,109.2333,
15,Camacari,BR,78.57,few clouds,-12.6975,-38.3242,
21,Bambous Virieux,MU,82.65,broken clouds,-20.3428,57.7575,
23,Busselton,AU,79.09,few clouds,-33.6500,115.3333,
27,Raga,SS,84.70,clear sky,8.4596,25.6780,
29,Liloy,PH,83.19,heavy intensity rain,8.1144,122.6594,
37,Lilongwe,MW,82.60,clear sky,-13.9669,33.7873,
40,Atuona,PF,78.17,few clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5,000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,Paradise Inn
8,Hervey Bay,AU,82.04,clear sky,-25.2986,152.8535,Shelly Bay Resort
11,Qui Nhon,VN,81.25,few clouds,13.7667,109.2333,Hoang Yen Canary Hotel
15,Camacari,BR,78.57,few clouds,-12.6975,-38.3242,Hotel Cambuci
21,Bambous Virieux,MU,82.65,broken clouds,-20.3428,57.7575,Casa Tia Villa
23,Busselton,AU,79.09,few clouds,-33.6500,115.3333,Observatory Guest House
27,Raga,SS,84.70,clear sky,8.4596,25.6780,
29,Liloy,PH,83.19,heavy intensity rain,8.1144,122.6594,House of Villarin
37,Lilongwe,MW,82.60,clear sky,-13.9669,33.7873,Spendwise furnished apartments.
40,Atuona,PF,78.17,few clouds,-9.8000,-139.0333,Villa Enata


In [9]:
# 7. Drop the rows where there is no Hotel Name
hotel_df.replace("", np.nan, inplace = True) 
hotel_df.dropna(subset = ["Hotel Name"], inplace = True)

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(15.0, 7.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))